In [1]:
import scanpy as sc
import pyro
import sys
sys.path.append("../")

## Example 4: Echidna & Pyro

In this tutorial, we show you how to use Echidna more flexibly by accessing the underlying data in the model. This will be useful if you have prior familiarity with Pyro and aim to perform more custom work.

In [2]:
import echidna as ec
ec.pl.activate_plot_settings()

As in the previous example, we load our AnnData from our previous run. Keep in mind that the copy number data is now saved in `.obs`.

In [3]:
adata = sc.read_h5ad("data/F01_on_echidna.h5")

Loading a model reads in a pickled model object and loads the Pyro parameter store. We can see that this is true by looking at the keys in the global parameter store.

In [4]:
echidna = ec.tl.load_model(adata)

In [5]:
pyro.get_param_store().keys()

dict_keys(['eta_mean', 'c_shape', 'scale_shape', 'scale_rate', 'corr_loc', 'corr_scale'])

If you make changes to the model object or param store, you can save those changes with the following function:

In [6]:
ec.tl.save_model(adata, echidna, overwrite=False)

2024-07-24 03:04:03,272 | INFO : Saving echidna model with run_id 20240724-030403.


The `echidna` object will contain the configuartion, the model and guide functions to run a forward pass through the model, as well as Torch tensors for the ground truth $\eta$, $c$ and $\Sigma$ from training.

In [7]:
eta, c, cov = echidna.eta_ground_truth, echidna.c_ground_truth, echidna.cov_ground_truth

Let's also see how we build the data tensors to do a forward pass. Echidna does this under-the-hood for you for safety purposes, but it may be helpful to do have access to the actual tensors. 

In [8]:
data = ec.tl.build_torch_tensors(adata, echidna.config)

A pass of the data through the model will return the X and W tensors passed through the model, after a full sampling of the model.

In [9]:
echidna.model(*data)

(tensor([[  4.,   2.,   0.,  ..., 217.,   0.,   2.],
         [  4.,   1.,   0.,  ..., 312.,   0.,   4.],
         [  4.,   2.,   1.,  ..., 397.,   1.,   7.],
         ...,
         [  0.,   0.,   0.,  ...,   2.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   3.,   0.,   0.],
         [  1.,   0.,   0.,  ...,   5.,   0.,   0.]], device='cuda:0'),
 tensor([1., 1., 1.,  ..., 3., 3., 3.], device='cuda:0'))

In [10]:
echidna.guide(*data)

This function uses poutine to trace the guide with the data, replay the model with that guide, and finally return nodes from the trace of the replay with the data. See for example a sampled $\eta$ compared to the ground truth of many averaged samples.

In [11]:
learned_params = ec.tl.get_learned_params(echidna, data)

In [12]:
learned_params["eta"]["value"], eta

(tensor([[ 2.4991e-01, -1.4804e-01, -7.3896e-01,  ..., -1.2036e+00,
          -1.7756e-01, -2.6762e+00],
         [ 1.5998e-01,  2.1188e-01,  1.4875e-01,  ..., -4.9528e-01,
           7.6729e-04, -2.7873e-01],
         [ 4.9852e-01,  3.6710e-01, -9.1138e-02,  ..., -1.8437e+00,
           8.9070e-01, -8.3446e-01],
         ...,
         [ 2.1767e+00,  2.3594e+00,  2.3668e+00,  ...,  2.3188e+00,
           2.4001e+00,  2.7859e+00],
         [ 2.1418e+00,  2.3149e+00,  1.8691e+00,  ...,  2.4096e+00,
           4.3547e-01,  1.0031e+00],
         [ 2.2209e+00,  2.1759e+00,  2.2528e+00,  ...,  1.4022e+00,
          -4.2894e-01,  1.3390e-01]], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([[0.9113, 0.7697, 0.7593,  ..., 1.9952, 2.3261, 2.2024],
         [0.7013, 0.6799, 0.7575,  ..., 2.2563, 2.2787, 2.1863],
         [0.4156, 0.7305, 0.6524,  ..., 2.4473, 1.8369, 1.9346],
         ...,
         [0.2455, 0.6595, 0.6825,  ..., 2.7323, 2.3762, 1.7365],
         [0.9083, 0.5716, 1.5581,  ...,

The rest of the problem, as they say, is behind the keyboard.